In [1]:
import os
import glob
import numpy as np
import time
from helper import R_to_angle
from params import par
from torchvision import transforms
from PIL import Image
import torch
import math

In [2]:
train_video = ['00', '02', '08', '09', '06', '04', '10']
# train_video = ['00', '02', '08' ]
image_path_list = []
for folder in train_video:
    image_path_list += glob.glob('KITTI/images/{}/*.png'.format(folder))

In [4]:
# image_path_list

In [5]:
len(image_path_list)

17437

In [6]:
img_as_img = Image.open(image_path_list[10])

In [9]:
to_tensor = transforms.ToTensor()

In [13]:
img_as_tensor = to_tensor(img_as_img)
img_as_tensor -= 0.5

In [15]:
img_as_tensor.max()

tensor(0.5000)

In [16]:
img_as_tensor.min()

tensor(-0.5000)

In [22]:
img_as_tensor.shape

torch.Size([3, 376, 1241])

In [19]:
img_as_np = np.array(img_as_img)
img_as_np = np.rollaxis(img_as_np, 2, 0)

In [21]:
img_as_np.shape

(3, 376, 1241)

In [23]:
img_as_np.max()

255

In [24]:
img_as_np.min()

0

In [29]:
img_as_tensor.std()

tensor(0.3237)

In [31]:
use_cuda = True
device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")


In [41]:
def denorm(batch, mean=[-0.0810, 0.1, 0.1], std=[0.3237, 0.4 , 0.4]):
    """
    Convert a batch of tensors to their original scale.

    Args:
        batch (torch.Tensor): Batch of normalized tensors.
        mean (torch.Tensor or list): Mean used for normalization.
        std (torch.Tensor or list): Standard deviation used for normalization.

    Returns:
        torch.Tensor: batch of tensors without normalization applied to them.
    """
    if isinstance(mean, list):
        mean = torch.tensor(mean).to(device)
    if isinstance(std, list):
        std = torch.tensor(std).to(device)

    return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)

In [43]:
denorm(img_as_tensor
       ).shape

torch.Size([1, 3, 376, 1241])